## Importing required libraries

In [17]:
from tqdm import tqdm
import logging
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split


from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import GloVe, vocab
from torchtext.datasets import AG_NEWS
from torchtext.data.functional import to_map_style_dataset


from torchdata.datapipes.iter import IterableWrapper, Mapper

from gensim.models import Word2Vec

import seaborn as sns
from sklearn.manifold import TSNE
from IPython.core.display import display, SVG

%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

C:\Users\vijay\AppData\Local\Temp\ipykernel_19496\2207896738.py:29: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, SVG


#### Defing a funtion to plot word embeddings in 2d space

In [19]:
def plot_embeddings(word_embeddings, vocab=vocab):

    # performing t-SNE on the embeddings to reduce their dimentionality to 2D using "TSNE" function from "sklearn" libary
    tsne = TSNE(n_components=2, random_state=0)
    word_embeddings_2d = tsne.fit_transform(word_embeddings)

    # plotting the results with labels from vocab
    plt.figure(figsize=(15,15))
    for i, word in enumerate(vocab.get_itos()):
        plt.scatter(word_embeddings_2d[i,0], word_embeddings_2d[i,1])
        plt.annotate(word, (word_embeddings_2d[i,0], word_embeddings_2d[i,1]))

    plt.xlabel("t-SNE component 1")
    plt.ylabel("t-SNE component 2")
    plt.title("Word Embeddings visualized with t-SNE")
    plt.show()

#### Defining a function to return similar words to a given word by calculating Cosine distance~

In [20]:
def find_similar_word(word, word_embeddings, top_k=5):
    if word not in word_embeddings:
        print("Word not found in embeddings.")
        return []

    target_embedding = word_embeddings[word]

    # calculating cosine distange with all words
    similarities = {}
    for w, embedding in word_embeddings.items():
        if w != word:
            similarity = torch.dot(target_embedding, embedding) / (torch.norm(traget_embedding) * torch.norm(embedding))
            similarities[w] = similarity.item()

    # soting the similarities
    sorted_similarities = sorted(similarities.items(), key= lambda x: x[1], reverse=True)

    # returning the tok k similar words
    most_similar_word = [w for w, _ in sorted_similarities[:tok_k]]
    return most_similar_words